# Goal:
由于健康模型在高胆数据上预测准确率低，需要从数据的角度上描述（健康、高胆换血、高胆未换血）的差异。
1. 找到论文里一般用的特征有哪些，做差异分析
2. 用这些特征建立机器学习模型，看feature importance，比较哪个特征贡献大。


In [1]:
import os
import copy
import pylab
import numpy as np
import pandas as pd
import matplotlib
#matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import mne 
from multiprocessing import Pool
import threading
import pingouin as pg
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
mne.set_log_level('ERROR')

In [2]:
import matplotlib.font_manager
matplotlib.rcParams['savefig.dpi'] = 300
matplotlib.rcParams["figure.dpi"] = 100
plt.rcParams['font.sans-serif']=['SimHei'] #Show Chinese label
plt.rcParams['axes.unicode_minus']=False   #These two lines need to be set manually
#%matplotlib tk
import inspect
inspect.getfullargspec(mne.io.read_raw_edf)

FullArgSpec(args=['input_fname', 'eog', 'misc', 'stim_channel', 'exclude', 'infer_types', 'include', 'preload', 'units', 'encoding'], varargs=None, varkw=None, defaults=(None, None, 'auto', (), False, None, False, None, 'utf8'), kwonlyargs=['verbose'], kwonlydefaults={'verbose': None}, annotations={})

In [3]:
def readPath(path):
    file_path = []
    for root, dirs, files in os.walk(path):
        
        for file in files:
            if file.endswith('.edf'):
                path_name = os.path.join(root, file)
                file_path.append(path_name)
            #print(path_name)

    return file_path

In [4]:
def loadFile(path, exclude_channels=True, 
                             crop_wake_mins=30):
    """Load a raw.edf file.

    Parameters
    ----------
    path : str
        Path to the .edf file containing the raw data. 
    exclude_channels : bool
        If True, only keep EEG channels and discard other modalities 
        (speeds up loading).
    crop_wake_mins : float
        Number of minutes of wake events before and after sleep events.

    Returns
    -------
    mne.io.Raw :
        Raw object containing the EEG and annotations.        
    """

    mapping = {
    # 'EEG Fp1-AV',
    # 'EEG Fp2-AV',
     'EEG C3-AV',
     'EEG C4-AV',
    # 'EEG P3-AV',
     'EEG O1-AV',
     'EEG T3-AV',
     'EEG T4-AV',
     'EEG Cz-AV',
     'EEG Pz-AV',
     'EEG O2-AV',
    # 'EEG P4-AV',
     'ECG',
     'EMG Left_Leg',
     'EMG Right_Leg'}

    
    exclude = mapping if exclude_channels else ()

    #Read the raw data and annotations
    raw = mne.io.read_raw_edf(path, exclude=exclude)

    if not exclude_channels:
        raw.set_channel_types(mapping)

    # Rename EEG channel: replace EEG
    ch_names = {i: i.replace('EEG ', '') 
                for i in raw.ch_names if 'EEG' in i}
    mne.rename_channels(raw.info, ch_names)


    #store the subject information:
    
    basename = os.path.basename(path)
    subj_nb = os.path.splitext(basename)[0] #e.g., 113
    raw.info['subject_info'] = {'id': subj_nb}
    

    return raw

In [5]:
path_1 = './healthy-2023'
path_2 = './healthy-2024'
path_3 = './hyper-bloodchange-2024'
path_4 = './hyper-blood-change-notchange-sample'

In [6]:
res_1 = readPath(path_1) #healthy - from 2023
res_4 = readPath(path_4) #高胆红素血症（换血及未换血各3个）

In [7]:
raws_healthy = [loadFile(f) for f in res_1] #healthy dataset
raws_hyper_6 = [loadFile(f) for f in res_4] #hyper: blood exchanged vs not (6)

The datasets we have:
- raws_healthy
- raws_hyper_26
- raws_hyper_6

# Filter

In [8]:
def filter(raw):
    l_freq, h_freq = None, 30
    raw.load_data().filter(l_freq, h_freq)  # filtering happens in-place 

In [9]:
for raw in raws_healthy:
    filter(raw)

In [10]:
for raw in raws_hyper_6:
    filter(raw)

# Extract epochs

In [11]:
def extract_epochs(raw, chunk_duration=30.):
    """Extract non-overlapping epochs from raw data.
    
    Parameters
    ----------
    raw : mne.io.Raw
        Raw data object to be windowed.
    chunk_duration : float
        Length of a window.
    
    Returns
    -------
    np.ndarray
        Epoched data, of shape (n_epochs, n_channels, n_times).
    np.ndarray
        Event identifiers for each epoch, shape (n_epochs,).
    """
    custom_mapping = {"清醒1": 3, "AS1":2, "QS1":1}
    events, _ = mne.events_from_annotations(
        raw, 
        event_id=custom_mapping, 
        chunk_duration=chunk_duration)


    tmax = 30. - 1. / raw.info['sfreq']  # tmax in included
    picks = mne.pick_types(raw.info, eeg=True)
    epochs = mne.Epochs(raw=raw, events=events, picks=picks, preload=True,
                        event_id=custom_mapping, tmin=0., tmax=tmax, baseline=None)
    
    return epochs

### Get epoch lists for datasets with annotations:
Datasets:
- raws_healthy
- raws_hyper_6

Output epochs:
- epochs_healthy
- epochs_hyper_6

In [12]:
#对MyThread进行封装
class MyThread(threading.Thread):
    def __init__(self,func,args,name=''):
        threading.Thread.__init__(self)
        self.func = func
        self.name = name
        self.args = args
        self.res = None
    def run(self):
        self.res = self.func(self.args)
    def getResult(self):
        return self.res

In [13]:
if __name__ == '__main__' :
    Threads = []
    epochs_healthy = []
    i = 0
    for raw in raws_healthy:
        t = MyThread(func=extract_epochs,args=raw,name ='Thread'+ str(i))
        i+=1
        Threads.append(t)
        t.start()
    for t in Threads:
        t.join()
        epochs_healthy.append(t.getResult()) #epochs_all would be the dataset of all epochs

In [14]:
if __name__ == '__main__' :
    Threads = []
    epochs_hyper_6 = []
    i = 0
    for raw in raws_hyper_6:
        t = MyThread(func=extract_epochs,args=raw,name ='Thread'+ str(i))
        i+=1
        Threads.append(t)
        t.start()
    for t in Threads:
        t.join()
        epochs_hyper_6.append(t.getResult()) #epochs_all would be the dataset of all epochs

#### Frequency analysis: Sleep stages x per subject x electrode positions

Based on the 10-20 international system of electrode placement and the electodes we have: <br>
Our electrodes can be arranged into: <br>
- Frontal lobe: FP1, FP2 (picks: 0,1)
- Parietal lobe: C3, Cz, C4, P3, Pz, P4 (picks: 2,10,3,4,11,5)
- Temporal lobe: T7, T8 (picks: 8,9)
- Occipital lobe: O1, O2 (picks: 6,7)

Next, we can plot epochs of each sleep stage as an image map: <br>
1. each row of pixels in the image representing a single epoch
2. the horizontal axis representing time
3. each pixel's color representing the signal value at that time sample for that epoch

We saved the image maps.

## Power band datasets

In [15]:
def get_powerbands(epochs,picks,event):
    """
    Parameters
    ------------
    epochs : Epochs
        The data
    picks : Channels
        Channels that uses for analys
    event : sleep stage
        Sleep stage uses for analys

    Returns
    -----------
    subplots
    """
    # specific frequency bands
    FREQ_BANDS = {
        "delta": [0.5, 4.5],# qs
        "theta": [4.5, 8.5],# as
        "alpha": [8.5, 11.5],
        "sigma": [11.5, 15.5],
        "beta": [15.5, 30],
    }
    spectrum = epochs[event].compute_psd(picks=picks,fmin=0.5,fmax=30.0)
    psds,freqs = spectrum.get_data(return_freqs=True)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    power_bands = []
    index = 0
    for fmin, fmax in FREQ_BANDS.values():
        #psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        psds_band = np.median(psds[:, :, (freqs >= fmin) & (freqs < fmax)],axis=-1)
        power_bands.append(psds_band.reshape(len(psds), -1))
    return power_bands

In [16]:
healthy_awake = []
hyper6_awake = []
for i in range(6):
    healthy_awake.append(get_powerbands(epochs_healthy[i],['Fp1-AV','Fp2-AV'],'清醒1'))
    hyper6_awake.append(get_powerbands(epochs_hyper_6[i],['Fp1-AV','Fp2-AV'],'清醒1'))

In [17]:
healthy_as = []
hyper6_as = []
for i in range(6):
    healthy_as.append(get_powerbands(epochs_healthy[i],['Fp1-AV','Fp2-AV'],'AS1'))
    hyper6_as.append(get_powerbands(epochs_hyper_6[i],['Fp1-AV','Fp2-AV'],'AS1'))

In [18]:
healthy_qs = []
hyper6_qs = []
for i in range(6):
    healthy_qs.append(get_powerbands(epochs_healthy[i],['Fp1-AV','Fp2-AV'],'QS1'))
    hyper6_qs.append(get_powerbands(epochs_hyper_6[i],['Fp1-AV','Fp2-AV'],'QS1'))

In [19]:
len(healthy_awake[0][0][3])

2

In [525]:
healthy_qs_all = []
for healthy in epochs_healthy:
    healthy_qs_all.append(get_powerbands(healthy,['Fp1-AV','Fp2-AV'],'QS1'))

## Combine healthy and hyper datasets as a dataframe

In [20]:
# waves = ['delta','theta','alpha','sigma','beta']

# def dataframe_powerbands(healthy_powerband, hyper6_powerband, picks_num):
#     """
#     Parameters
#     ----------------
#     healthy_dataset: powerband datasets of all healthy subjects
#     hyper6_dataset: powerband datasets of hyperbilirubin subjects
#     picks_num: the num of pick channel, this can be 0,1, which represented 'Fp1-AV','Fp2-AV'

#     Returns
#     -----------------
#     df: a dataframe includes 
#     status (healthy/换血/未换血), 
#     wave ('delta','theta','alpha','sigma','beta'), 
#     psd values of all epochs - each value is the mean psd of a epoch
#     """
#     status,wave,psd_set = [],[],[]
    
#     for i in range(len(waves)):
#         for j in range (12):
#             if j<6 :
#                 #calculate healthy_powerband
#                 status.append( "健康")
#                 wave.append(waves[i])
#                 psd_set.append(healthy_powerband[j][i][:,picks_num]) #channel selection: 0,1
               
#             elif j>=6 and j<9:
#                 #calculate hyper6_powerband
#                 status.append( "换血")
#                 wave.append(waves[i])
#                 psd_set.append(hyper6_powerband[j-6][i][:,picks_num])
               
#             else:
#                 #calculate hyper6_powerband
#                 status.append( "未换血")
#                 wave.append(waves[i])
#                 psd_set.append(hyper6_powerband[j-6][i][:,picks_num])
                
    
#     df= pd.DataFrame({'status':status,'wave':wave,'psd_set':psd_set})         
#     return df

In [21]:
# df_awake = dataframe_powerbands(healthy_awake, hyper6_awake, picks_num = 0)

In [22]:
# df_as = dataframe_powerbands(healthy_as, hyper6_as, picks_num = 0)

In [23]:
# df_qs = dataframe_powerbands(healthy_qs, hyper6_qs, picks_num = 0)

## statistical analysis

In [24]:
# ranksums(x.iloc[0]['psd_mean'], x.iloc[1]['psd_mean'])

In [25]:
# for i in range(60):
#     for j in range(60):
#         print(j)
#         print(ranksums(x.iloc[i]['psd_mean'], x.iloc[j]['psd_mean']))

In [26]:
# #generate a combined dataframe for Awake
# df_awake = dataframe_powerbands(healthy_awake, hyper6_awake, sleep_stage='Awake', picks_num = 0)

## ranksum test

In [43]:

# waves = ['delta','theta','alpha','sigma','beta']
# psd_waves_subject = []
# for i in range(len(waves)):
#     psd_for_waves = []
#     for j in range(len(healthy_awake)):
#         psd_values = healthy_awake[j][i][:, 0] #0 is the channel number
#         psd_for_waves.append(psd_values)
#     #print(psd_for_waves)
#     psd_waves_subject.append(psd_for_waves)
#     #print(ranksums(psd_for_waves[0], psd_for_waves[1]))

In [44]:
# len(psd_waves_subject[0])

In [45]:
# for x in range(len(waves)):
#     for y in range(len(healthy_awake)):
#         z = y+1
#         while z < len(healthy_awake):
#             #print(waves[x])
#             #print(y)
#             print(ranksums(psd_waves_subject[x][y],psd_waves_subject[x][z] ))
            
#             z+=1
            

In [116]:
from scipy.stats import ranksums
from scipy.stats import mannwhitneyu
waves = ['delta','theta','alpha','sigma','beta']

def get_psd_all(healthy_powerband, hyper6_powerband, picks_num):
    psd = []

    for i in range(len(waves)):
        psd_set = []
        for j in range (12):
            if j<6 :
                #calculate healthy_powerband
                psd_set.append(healthy_powerband[j][i][:,picks_num]) #channel selection: 0,1
               
            elif j>=6 and j<9:
                #calculate hyper6_powerband
                psd_set.append(hyper6_powerband[j-6][i][:,picks_num])
               
            else:
                #calculate hyper6_powerband
                psd_set.append(hyper6_powerband[j-6][i][:,picks_num])    
            
        psd.append(psd_set)
                   
    return psd

Change the input dataset here.

In [489]:
psd = get_psd_all(healthy_qs, hyper6_qs, 0)

## Perform the Shapiro-Wilk test for normality

The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution. <br>


In [490]:
all_psd = []
for i in range(len(waves)):    
    for j in range(len(psd[0])):
        res = stats.shapiro(psd[i][j])
        all_psd.append(res)

In [491]:
normal_psd = [] #normally distributed
not_normal_psd = []
for i in range(len(waves)):    
    for j in range(len(psd[0])):
        if stats.shapiro(psd[i][j])[1] > 0.05:
            normal_psd.append(stats.shapiro(psd[i][j]))
        else:
            not_normal_psd.append(stats.shapiro(psd[i][j]))

In [492]:
len(all_psd),len(normal_psd)

(60, 7)

In [493]:
normal_psd

[ShapiroResult(statistic=0.9940241033105613, pvalue=0.8896105833311919),
 ShapiroResult(statistic=0.9769712755715713, pvalue=0.0619713018269692),
 ShapiroResult(statistic=0.9915160971171525, pvalue=0.20876604266798915),
 ShapiroResult(statistic=0.994470893029101, pvalue=0.17819294191157747),
 ShapiroResult(statistic=0.988722411789403, pvalue=0.09544678843849554),
 ShapiroResult(statistic=0.9921482934484612, pvalue=0.3114243383371289),
 ShapiroResult(statistic=0.9854906487401484, pvalue=0.22249319511622795)]

Only 7 out of 60 psds, is normally distributed. Then we cannot use t test, we should choose non parametric test.

## Non-parametric test

Perform the Mann-Whitney U rank test on two independent samples. <br>
The Mann-Whitney U test is a nonparametrix test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y.

In [494]:
wave = []
subject_1 = []
subject_2 = []
statistic = []
p_value = []

for i in range(len(waves)):  
    for j in range(len(psd[0])):       
        k = j+1
        while k < len(psd[0]):
            wave.append(waves[i])
            subject_1.append(j+1)
            subject_2.append(k+1)
            # print(waves[i])
            # print(j)
            #print(ranksums(awake_psd[i][j],awake_psd[i][k]))
            statistic.append(mannwhitneyu(psd[i][j],psd[i][k])[0])
            p_value.append(mannwhitneyu(psd[i][j],psd[i][k])[1])
            k+=1

df = pd.DataFrame({'wave':wave, 'subject_1':subject_1, 'subject_2':subject_2, 'statistic':statistic, 'p_value':p_value})
pd.set_option('display.max_rows', None)

In [495]:
df.loc[df['subject_1'] <=6, 'subject_1_status'] = '健康'
df.loc[(df['subject_1'] >6) & (df['subject_1'] <=9) , 'subject_1_status'] = '换血'
df.loc[df['subject_1'] >9 , 'subject_1_status'] = '未换血'

In [496]:
df.loc[df['subject_2'] <=6, 'subject_2_status'] = '健康'
df.loc[(df['subject_2'] >6) & (df['subject_2'] <=9) , 'subject_2_status'] = '换血'
df.loc[df['subject_2'] >9 , 'subject_2_status'] = '未换血'

In [520]:
df

,wave,subject_1,subject_2,statistic,p_value,subject_1_status,subject_2_status
0,delta,1,2,5034.0,4.169464e-29,健康,健康
1,delta,1,3,16725.0,5.834356e-01,健康,健康
2,delta,1,4,4005.0,1.338199e-28,健康,健康
3,delta,1,5,3626.0,5.725181e-24,健康,健康
4,delta,1,6,7327.0,2.665852e-07,健康,健康
5,delta,1,7,40904.0,6.356545e-69,健康,换血
6,delta,1,8,6127.0,1.517564e-55,健康,换血
7,delta,1,9,7816.0,1.382995e-79,健康,换血
8,delta,1,10,11070.0,1.162427e-32,健康,未换血
9,delta,1,11,3217.0,1.580433e-55,健康,未换血


### Different distribution: reject the null hypothesis of same distribution

In [519]:
diff_df = df[df['p_value']>0.05]
diff_df

,wave,subject_1,subject_2,statistic,p_value,subject_1_status,subject_2_status
1,delta,1,3,16725.0,0.583436,健康,健康
12,delta,2,4,6317.0,0.846602,健康,健康
13,delta,2,5,5669.0,0.513982,健康,健康
18,delta,2,10,14355.0,0.059051,健康,未换血
20,delta,2,12,12611.0,0.745784,健康,未换血
30,delta,4,5,5088.0,0.345246,健康,健康
37,delta,4,12,11161.0,0.869481,健康,未换血
42,delta,5,10,9973.0,0.395629,健康,未换血
44,delta,5,12,8835.0,0.352690,健康,未换血
51,delta,7,8,110610.0,0.405310,换血,换血


In [518]:
print('The number of all tested subjects is: '+ str(len(df)))
print('The number of all statistically different subjects is: '+ str(len(diff_df)))
print('the different ratio is: '+str(round(len(diff_df)/len(df),2)))

The number of all tested subjects is: 330
The number of all statistically different subjects is: 286
the different ratio is: 0.87


In [516]:
330/5

66.0

### 分波段分析

### 比例统计：
- 健康 VS 健康
- 健康 VS 换血
- 健康 VS 未换血
- 换血 VS 未换血

In [500]:
#('delta','theta','alpha','sigma','beta')
selected_wave = 'beta'
diff_df = df[df['p_value']>0.05]

diff_df = diff_df[diff_df['wave'] == selected_wave]

diff_df[['wave','subject_1_status', 'subject_2_status']] = np.sort(diff_df[['wave', 'subject_1_status', 'subject_2_status']], axis=1)
#diff_df.groupby(['subject_1_status', 'subject_2_status']).statistic.agg('count').reset_index().values.tolist()

diff_count = diff_df.groupby(['subject_1_status', 'subject_2_status']).statistic.agg('count').reset_index()
diff_count.insert(loc=3,column='ratio', value=round(diff_count['statistic']/diff_count['statistic'].sum(),2))
diff_count.insert(loc=2, column='pair', value =diff_count['subject_1_status'] + ' VS ' + diff_count['subject_2_status'] )


diff_count = diff_count.style.set_caption('Count of different datasets in QS, beta')
diff_count

,subject_1_status,subject_2_status,pair,statistic,ratio
0,健康,健康,健康 VS 健康,1,0.120000
1,健康,换血,健康 VS 换血,4,0.500000
2,健康,未换血,健康 VS 未换血,3,0.380000


In [521]:
len(diff_df)

44

In [501]:
diff_df

,wave,subject_1,subject_2,statistic,p_value,subject_1_status,subject_2_status
292,beta,3,11,9892.0,0.686018,健康,未换血
294,beta,4,5,5072.0,0.366478,健康,健康
297,beta,4,8,11484.0,0.465478,健康,换血
299,beta,4,10,12179.0,0.195985,健康,未换血
304,beta,5,8,9149.0,0.174174,健康,换血
306,beta,5,10,9700.0,0.651498,健康,未换血
309,beta,6,7,41419.0,0.324771,健康,换血
311,beta,6,9,15874.0,0.897361,健康,换血


In [498]:
# fig, ax = plt.subplots()
# ax.pie(diff_count['ratio'],labels = diff_count['pair'], autopct='%1.1f%%')
# #ax.pie(diff_count['statistic'],labels = diff_count['pair'])
# ax.set_title('AS delta')